In [790]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [791]:
# TRAIN
# Preparedata
Clss_tr = []
Words_tr = []
Exclude = ["I", "B", "OBRN_B", "MEA_BI", "DDEM", "ABB", 'B_NAME']
for file in os.listdir("../input/lst20corpus/LST20_Corpus/train"):
    f = open("../input/lst20corpus/LST20_Corpus/train/" + file, "r")
    x = f.read()
    for long in x.split("\n"):
        if long == '':
            pass
        else :
            Wrd = long.split("\t")[0]
            Cls = long.split("\t")[2]
            if Cls in Exclude :
                pass
            if Cls == "ORG_I":
                Words_tr.append(Wrd)
                Clss_tr.append("I_ORG")
            if Cls == "PER_I":
                Words_tr.append(Wrd)
                Clss_tr.append("I_PER")
            if Cls == "B_NAME":
                Words_tr.append(Wrd)
                Clss_tr.append("B_PER")
            if Cls in Exclude:
                pass
            if Wrd in '-------------------------------------' :
                pass
            else:
                Words_tr.append(long.split("\t")[0])
                Clss_tr.append(long.split("\t")[2])

In [792]:
Clss_e = []
Words_e = []
for file in os.listdir("../input/lst20corpus/LST20_Corpus/eval"):
    f = open("../input/lst20corpus/LST20_Corpus/eval/" + file, "r")
    x = f.read()
    for long in x.split("\n"):
        if long == '':
            pass
        else :
            if long.split("\t")[2] in Exclude :
                pass
            if long.split("\t")[2] == "LOC_I":
                Words_e.append(long.split("\t")[0])
                Clss_e.append("I_LOC")
            if long.split("\t")[0] in '-------------------------------------' :
                pass
            if long.split("\t")[2] == "ORG_I":
                Words_e.append(long.split("\t")[0])
                Clss_e.append("I_ORG")
            if long.split("\t")[2] == "PER_I":
                Words_e.append(long.split("\t")[0])
                Clss_e.append("I_PER")
            if long.split("\t")[2] == "B_NAME":
                Words_e.append(long.split("\t")[0])
                Clss_e.append("B_PER")
            else:
                Words_e.append(long.split("\t")[0])
                Clss_e.append(long.split("\t")[2])
                

Clss_t1 = []
Words_t1 = []
Sent_t1 = []
sent1 = []
idx1 = 0
for file in os.listdir("../input/lst20corpus/LST20_Corpus/test"):
    f = open("../input/lst20corpus/LST20_Corpus/test/" + file, "r")
    x = f.read()
    for long in x.split("\n"):
        if long == '' :
            pass
        else :
            if long.split("\t")[2] in Exclude :
                pass
            if long.split("\t")[0] in '-------------------------------------' :
                pass
            if long.split("\t")[2] == "ORG_I":
                Words_t1.append(long.split("\t")[0])
                Clss_t1.append("I_ORG")
            if long.split("\t")[2] == "PER_I":
                Words_t1.append(long.split("\t")[0])
                Clss_t1.append("I_PER")
            if long.split("\t")[2] == "B_NAME":
                Words_t1.append(long.split("\t")[0])
                Clss_t1.append("B_PER")
            else:
                idx1 += 1
                Words_t1.append(long.split("\t")[0])
                Clss_t1.append(long.split("\t")[2])
                sent1.append(long.split("\t")[0])
                if idx1%3 == 0 and idx1 > 1:
                    Sent_t1.append(sent1)
                    sent1 = []

In [793]:
Words = Words_tr + Words_e
Clss = Clss_tr + Clss_e 

In [794]:
len(Words)

2950993

In [795]:
from collections import Counter

# Create count trasition dict (for hidden state)     
Trasition = [Clss[idx] + "_" + Clss[idx+1] for idx in range(len(Clss)-1)]

# Create count emission dict
Emission = [Clss[idx] + "_" + Words[idx] for idx in range(len(Clss))]

Count_Words     = dict(Counter(Words))
Count_Clss      = dict(Counter(Clss))
Count_Trasition = dict(Counter(Trasition))
Count_Emission  = dict(Counter(Emission))

Clss_list = list(set(Clss))

In [796]:
def Clss_prob(Count_Clss, Clss, clss):
    if Count_Clss[clss] / len(Clss) == 0 :
        return 0
    else:
        prob_score = -np.log( Count_Clss[clss] / len(Clss))
        return 1/(prob_score + 1)

def Transition_prob(Count_Trasition, Trasition, Clss1, Clss2):
    if Clss1+"_"+ Clss2 not in Count_Trasition:
        return 0
        
    elif Count_Trasition[Clss1+"_"+ Clss2] / len(Trasition)  == 0 :
        return 0
    else:
        prob_score = -np.log( Count_Trasition[Clss1+"_"+ Clss2] / len(Trasition) )
        return 1 / (prob_score + 1)
        
def Emission_prob(Count_Emission, Count_Words, Hidden, Word):
    if Word in Count_Words:
        if Hidden +"_"+ Word not in Count_Emission:
            Count_Emission.update({str(Hidden +"_"+ Word):0})
    if Word not in Count_Words:
        Count_Words.update({str(Word):1})
        Count_Emission.update({str(Hidden +"_"+ Word):1})
    
    if Count_Emission[Hidden +"_"+ Word]/Count_Words[Word] == 0:
        return 0
    else :
        prob_score = -np.log(Count_Emission[str(Hidden) + "_" + str(Word)] / Count_Words[Word])
    return 1 / (prob_score + 1)

In [797]:
# Applying Hidden Markov Model (HMM)

class Hidden_Markov_Model() :
    def __init__(self, Clss_list, Clss, Count_Clss, Count_Emission, 
                 Count_Trasition, Trasition, Count_Words, Sentence):
        self.Clss_list = Clss_list
        self.Clss      = Clss
        self.Count_Clss = Count_Clss
        self.Count_Emission = Count_Emission
        self.Count_Trasition = Count_Trasition
        self.Trasition = Trasition
        self.Count_Words = Count_Words
        self.Sentence  = Sentence
        
    def Viterbi_layer(self, iteration, Maxvalue, Previous_lab) :
        res1 = []
        P0_1 = []
        res2 = []
        P1_2 = []
        for clss1 in self.Clss_list :
            if iteration == 0 :
                p_hat = Clss_prob(self.Count_Clss, self.Clss, clss1)
                p_e1  = Emission_prob(self.Count_Emission, 
                                      self.Count_Words, 
                                      clss1, self.Sentence[iteration])
                res1.append(clss1)
                P0_1.append(p_hat * p_e1)
                
            elif iteration >= 1 :
                p_hat = Maxvalue
                p_t = Transition_prob(self.Count_Trasition, self.Trasition, 
                                      Previous_lab, clss1)
                
                p_e2 = Emission_prob(self.Count_Emission, self.Count_Words, 
                                      clss1, self.Sentence[iteration])
                res2.append(clss1)
                P1_2.append(p_hat * p_t * p_e2)
        
        if iteration == 0 :
            return res1[np.argmax(P0_1)], max(P0_1)
        if iteration >= 1 :
            return res2[np.argmax(P1_2)], max(P1_2)

In [798]:
Hidden_Markov = Hidden_Markov_Model(Clss_list, Clss, Count_Clss, Count_Emission, 
                                     Count_Trasition, Trasition, Count_Words, ['นาง', 'สุดารัตน์', ' '])
Pred = []
for idx in range(2):
    if idx == 0 :
        P = Hidden_Markov.Viterbi_layer(idx, Maxvalue = None, Previous_lab = None)
        Pred.append(P[0])
    if idx >= 1 :
        P = Hidden_Markov.Viterbi_layer(idx, Maxvalue = P[1], Previous_lab = P[0])
        Pred.append(P[0])

In [799]:
#Evaluation 
Pred1 = []
for sent in Sent_t1:
    Pred_sent1 = []
    Hidden_Markov = Hidden_Markov_Model(Clss_list, Clss, Count_Clss, Count_Emission, 
                                     Count_Trasition, Trasition, Count_Words, sent)
    for idx in range(3):
        if idx == 0 :
            P = Hidden_Markov.Viterbi_layer(idx, Maxvalue = None, Previous_lab = None)
            Pred_sent1.append(P[0])
        if idx >= 1 :
            P = Hidden_Markov.Viterbi_layer(idx, Maxvalue = P[1], Previous_lab = P[0])
            Pred_sent1.append(P[0])
    Pred1.append(Pred_sent1)

Prd = []
for prd in Pred1 :
    Prd += prd
    
# Evaluate data
sum((np.array([Prd[0:206714]]) == np.array(Clss_t1[0:206714]))[0])/len(Clss_t1)

0.8553102596512896

# **Submission**

In [801]:
pd.read_csv("../input/ss3-hackathon-online-natural-language-processing/ne_sample_submission.csv")

,Id,Predicted
0,1,B_TTL
1,2,B_PER
2,3,I_PER
3,4,NaN
4,5,NaN
...,...,...
69556,69557,NaN
69557,69558,NaN
69558,69559,NaN
69559,69560,NaN


In [802]:
f = open("../input/ss3-hackathon-online-natural-language-processing/ne_test.txt", "r")
Test = f.read()
Test.split("\n")

['นาง',
 'สุดารัตน์',
 '',
 'เกยุราพันธ์',
 '',
 'รมว.',
 'สาธารณสุข',
 '',
 'แถลง',
 'ถึง',
 'การ',
 'จัด',
 'งาน',
 'มหกรรม',
 'กิน',
 'ไก่',
 'ไทย',
 'ปลอด',
 'ภัย',
 '',
 '100',
 '%',
 '',
 'ว่า',
 '',
 'รัฐบาล',
 'ร่วม',
 'กับ',
 'กทม.',
 'จัด',
 'งาน',
 'ดัง',
 'กล่าว',
 'ขึ้น',
 'ที่',
 'ท้องสนามหลวง',
 'ใน',
 'วัน',
 'เสาร์',
 'ที่',
 '',
 '7',
 '',
 'ก.พ.',
 'นี้',
 '',
 'ตั้งแต่',
 '',
 '16.00',
 'น.',
 'เป็นต้นไป',
 '',
 'ส่วน',
 'พ.ต.ท.',
 'ทักษิณ',
 '',
 'ชินวัตร',
 'รัฐมนตรี',
 '',
 'จะ',
 'เดินทาง',
 'มา',
 'ถึง',
 'ที่',
 'งาน',
 'ใน',
 'เวลา',
 '',
 '16.30',
 'น.',
 '',
 'เพื่อ',
 'เยี่ยมชม',
 'นิทรรศการ',
 '',
 'และ',
 'ประกอบ',
 'อาหาร',
 'เมนู',
 '',
 'กระเพรา',
 'ไก่',
 'ไข่ดาว',
 'สุก',
 '',
 'ไก่',
 'ทอด',
 '',
 'ด้วย',
 'ตัว',
 'เอง',
 'และ',
 'แจกจ่าย',
 'ให้',
 'กับ',
 'ประชาชน',
 '',
 'โดย',
 'จะ',
 'มี',
 'นาย',
 'สมัคร',
 '',
 'สุนทรเวช',
 '',
 'ผู้',
 'ว่า',
 'ฯ',
 'กทม.',
 '',
 'และ',
 'ม.ล.',
 'ศิริเฉลิม',
 '',
 'สวัสดิวัฒน์',
 '',
 'หรือ',
 'หมึกแดง',
 

In [803]:
Words_Test[-1:]

['ขึ้น']

In [739]:
Words_Test = []
Sentence_Test = []
Sent = []
for idx, test in enumerate(Test.split("\n")[0:69561]):
    if idx%5 == 0 and (idx>=5) :
        Sentence_Test.append(Sent)
        Sent = []
    if idx == 69555 :
        Sentence_Test.append(Sent)
    if test == '':
        Words_Test.append("_")
        Sent.append("_")
    else :
        Sent.append(test)
        Words_Test.append(test)      

In [751]:
len(Sentence_Test)
Sentence_Test = Sentence_Test[0:13912]

In [758]:
Sentence_Test[-1]

['ให้', 'เกิด', 'เหตุ', 'ดัง', 'กล่าว']

In [772]:
# Test
Pred = []
for sent in Sentence_Test:
    Pred_sent = []
    Hidden_Markov = Hidden_Markov_Model(Clss_list, Clss, Count_Clss, Count_Emission, 
                                     Count_Trasition, Trasition, Count_Words, sent)
    for idx in range(5):
        if idx == 0 :
            P = Hidden_Markov.Viterbi_layer(idx, Maxvalue = None, Previous_lab = None)
            Pred_sent.append(P[0])
        if idx >= 1 :
            P = Hidden_Markov.Viterbi_layer(idx, Maxvalue = P[1], Previous_lab = P[0])
            Pred_sent.append(P[0])
    Pred.append(Pred_sent)

for idx in range(1):
    Hidden_Markov = Hidden_Markov_Model(Clss_list, Clss, Count_Clss, Count_Emission, 
                                     Count_Trasition, Trasition, Count_Words, ['ขึ้น'])
    if idx == 0 :
        P = Hidden_Markov.Viterbi_layer(idx, Maxvalue = None, Previous_lab = None)
        Pred.append([P[0]])
    if idx >= 1 :
        P = Hidden_Markov.Viterbi_layer(idx, Maxvalue = P[1], Previous_lab = P[0])
        Pred.append([P[0]])

In [779]:
Pred[-1]

['O']

In [774]:
#Submission
Real_pred = [] 
for prd in Pred:
    Real_pred += prd

In [784]:
len(Real_pred)

69561

In [786]:
### Submit 
import pandas as pd
DF_submit = pd.read_csv('../input/ss3-hackathon-online-natural-language-processing/ne_sample_submission.csv')
DF_submit

,Id,Predicted
0,1,B_TTL
1,2,B_PER
2,3,I_PER
3,4,NaN
4,5,NaN
...,...,...
69556,69557,NaN
69557,69558,NaN
69558,69559,NaN
69559,69560,NaN


In [787]:
DF_submit = pd.DataFrame(list(zip(list(DF_submit['Id']), list(Real_pred))), \
                              columns =['Id', 'Predicted'])
DF_submit

,Id,Predicted
0,1,O
1,2,B_PER
2,3,O
3,4,E_PER
4,5,O
...,...,...
69556,69557,O
69557,69558,O
69558,69559,O
69559,69560,O


In [788]:
DF_submit.to_csv('./DF_submit4.csv', index=False)